In [441]:

from Experiment import Experiment
import matplotlib.pyplot as plt
from Plotters import Plotters
import plotly.graph_objects as go
import plotly
import numpy as np
import time
import matplotlib.cm as colormap

%matplotlib qt

loadir = 'H:/My Drive/dark 2022/csv_dark' # dir to save the data frame in # H:\My Drive\Ronis Exp\sagiv\data_analysis
exp_name = ['manipulated_2023_08_06_40ms','manipulated_2023_08_09_80ms','manipulated_2023_08_10_100ms']#,'manipulated_2022_02_03_dark']

# ['manipulated_2022_02_03_dark',
#            'manipulated_2023_08_10_100ms',
#            'manipulated_2023_08_09_80ms',
#            'manipulated_2023_08_09_60ms',
#            'manipulated_2023_08_06_40ms',
#            'manipulated_2023_08_07_10ms',
#            'manipulated_2023_08_07_5ms',
# ] 



experiments = {exp:Experiment(loadir,exp) for exp in exp_name}
[exp.smooth_prop_movies('CM_real_x_body',['','_dot','_dot_dot'],'body')  for exp in experiments.values()]
[exp.project_prop_movies('CM_real_x_body_dot_smth')  for exp in experiments.values()]
[exp.project_prop_movies('CM_real_x_body_dot_dot_smth',header_name = 'CM_dot_dot')  for exp in experiments.values()]

min_max_v = [exp.min_max_point_movies('CM_dot_projected') for exp in experiments.values()]
min_max_pitch =[exp.min_max_point_movies('pitch_y_frame') for exp in experiments.values()]
zero_velocity = [exp.zero_velocity_movies('CM_dot_dot_projected') for exp in experiments.values()]
min_max_acc =[exp.min_max_point_movies('CM_dot_dot_projected') for exp in experiments.values()]

[exp.pqr_movies() for exp in experiments.values()]
# [exp.mean_mean_props_movies('phi_rw','phi_lw','mean_wing','mean_mean_phi') for exp in experiments]




[None, None, None]

In [437]:
plt.hist([v[v>50][0] for v in zero_velocity[0] if len(v[v>50])])


(array([2., 1., 2., 1., 8., 5., 4., 5., 4., 2.]),
 array([ 79.3125 ,  85.94375,  92.575  ,  99.20625, 105.8375 , 112.46875,
        119.1    , 125.73125, 132.3625 , 138.99375, 145.625  ]),
 <BarContainer object of 10 artists>)

In [388]:
from scipy.signal import find_peaks
min_max_v[0][0]

prop = 'CM_dot_dot_projected'
experiments_name = list(experiments.keys())

min_max_acc = [exp.min_max_point_movies('CM_dot_dot_projected',t0 = 15, t1 = 50) for exp in experiments.values()]
min_max_v = [exp.min_max_point_movies('CM_dot_projected',t0 = 15, t1 = 50) for exp in experiments.values()]
min_max_pitch = [exp.min_max_point_movies('pitch_body',t0 = 15, t1 = 50) for exp in experiments.values()]
min_max_pitch_dot = [exp.min_max_point_movies('pitch_body_dot',t0 = 15, t1 = 50) for exp in experiments.values()]

mov = experiments[experiments_name[0]].get_mov('mov106')
acc = mov.get_prop(prop,'body')[:,0]
time = mov.get_prop('time','body')[:,0]

max = find_peaks(acc, prominence=0.1 )
min = find_peaks(-acc, prominence=0.1 )


In [446]:
def get_max_min_prop(prop,mov,t1 = 50,t0 = 5):
    time = mov.get_prop('time','body')[:,0]
    idx_t1 = np.where(t1 == time)[0][0]
    idx_t0 = np.where(t0 == time)[0][0]
    if (t1 == False) | (t0 == False):
        acc = mov.get_prop(prop,'body')[:,0]
        time = mov.get_prop('time','body')[:,0]
        idx_t0 = 0
    else:


        acc = mov.get_prop(prop,'body')[idx_t0:idx_t1,0]
        time = mov.get_prop('time','body')[idx_t0:idx_t1,0]

    max = find_peaks(acc, prominence=0.1 )
    min = np.argmin(acc)
    return max[0] + idx_t0


color_map = colormap.datad["tab10"]['listed']

prop = 'pitch_y_frame_dot_dot'
prop = 'pitch_y_frame_dot_dot'
plt.figure(1)
fig,ax = plt.subplots(2,1)
experiments_name = list(experiments.keys())
for exp_idx,exp_name in  enumerate(experiments_name):
    min_max_pitch = [get_max_min_prop('CM_dot_dot_projected',experiments[exp_name].get_mov(movnmae)) for movnmae in  experiments[exp_name].mov_names]
    min_max_acc = [get_max_min_prop('CM_dot_dot_projected',experiments[exp_name].get_mov(movnmae),t1 = False,t0 = 10) for movnmae in  experiments[exp_name].mov_names]

    time_list = []
    time_list_min = []
    time2_list = []
    for idx,max in enumerate(min_max_pitch):
        if (len(max) > 0) & (len(min_max_acc[idx]) > 0) :
            
            mov_name = experiments[exp_name].mov_names[idx]
            mov = experiments[exp_name].get_mov(mov_name)
            time = mov.get_prop('time','body')[:,0]
            v_prop = mov.get_prop('CM_dot_projected','body')[:,0]
            max_prop = mov.get_prop('CM_dot_dot_projected','body')[:,0]
            min_idx = np.argmin(v_prop)
            mac = max_prop
            tm = time
            max_idx = np.argmax(mac)

            ax[0].plot(time,max_prop)
            ax[0].plot(time[max],max_prop[max],'*',color = 'black')
            ax[0].plot(tm[max_idx],mac[max_idx],'*',color = 'red')

            ax[1].plot(time,v_prop)
            ax[1].plot(time[min_idx],v_prop[min_idx],'*',color = 'blue')

            time_list += [time[max[0]]]
            time2_list += [time[min_max_acc[idx][-1]]]
            time_list_min += [time[min_idx]]
    plt.figure(1)
    plt.hist(time_list,alpha = 0.5,bins = 10, color = color_map[exp_idx])
    # plt.hist(time2_list,alpha = 0.5,bins = 10)
    plt.hist(time_list_min,alpha = 0.7,bins = 5, label =exp_name, color = color_map[exp_idx] )
plt.show()
plt.legend()

min_max

[array([ 719, 1717], dtype=int64),
 array([ 689, 1853, 2515, 3366, 4118], dtype=int64),
 array([2157, 3021, 3619], dtype=int64),
 array([ 753, 2217, 3119], dtype=int64),
 array([ 900, 2162, 2784], dtype=int64),
 array([ 657,  737, 1543, 1636, 1925, 2698], dtype=int64),
 array([2516, 3177], dtype=int64),
 array([ 512, 1444, 2310, 3123, 4437], dtype=int64),
 array([ 514, 1601, 2585, 3493], dtype=int64),
 array([ 709, 1720, 2683, 3442, 3981], dtype=int64),
 array([ 710, 1773, 2491, 3655], dtype=int64),
 array([ 728, 1791, 2657, 3262, 4176], dtype=int64),
 array([ 827, 1735, 2263, 3122, 3785, 4470], dtype=int64),
 array([ 626, 1808, 2374, 3107], dtype=int64),
 array([ 815, 1696, 2756, 3255, 4002], dtype=int64),
 array([ 641, 1867, 2373, 2635], dtype=int64),
 array([ 704, 2042, 2576, 2782], dtype=int64),
 array([ 727, 1853, 2660, 3421], dtype=int64),
 array([ 550, 1997], dtype=int64),
 array([ 937, 1974, 2863], dtype=int64),
 array([ 804, 1681, 2649], dtype=int64),
 array([1782, 2681], dtyp

In [380]:
min_max_acc[idx]

array([1097, 1378, 2039, 2445, 2879, 3336], dtype=int64)

In [198]:
plt.plot(time,acc)
[plt.plot(time[idx],acc[idx],'*') for idx in max[0]]
[plt.plot(time[idx],acc[idx],'*') for idx in min[0]]

[[<matplotlib.lines.Line2D at 0x148ba355c30>],

In [109]:
from sklearn.decomposition import PCA
import pandas as pd
from sklearn.cluster import KMeans

mov_feat = []
props = ['CM_dot_dot_projected','CM_dot_projected','pitch_y_frame_dot_dot']

exp_pd = []
for exp in list(experiments.values())[0:3]:
    mov_feat = []
    for idx,mov_name in enumerate(exp.mov_names):
        mov = exp.get_mov(mov_name)
        mov_feat.append(np.nanmean(mov.data['body'][15:60,:],axis = 0))

    featpd_exp = pd.DataFrame(data = mov_feat,columns = mov.header['body'])
    featpd_exp['exp_name'] = exp.experiment_name
    featpd_exp['mov'] = exp.mov_names
    exp_pd.append(featpd_exp)

feat = pd.concat(exp_pd,axis = 0).dropna().reset_index()


dataforpca = feat[props]
n_comp = 2
pca = PCA(n_components=n_comp)
principalComponents = pca.fit_transform(dataforpca)
principalDf = pd.DataFrame(data = principalComponents
             , columns = [f'pca_{idx}' for idx in range(n_comp)])

pca_name = [f'pca_{idx}' for idx in range(pca.n_components_)]


featpd = pd.concat([feat, principalDf], axis = 1)
kmeans = KMeans(n_clusters=3, random_state=0, n_init="auto").fit(featpd[pca_name])
print(kmeans.labels_)
pca.components_


C:\Users\Roni\AppData\Local\Temp\ipykernel_15164\3004972900.py:13: RuntimeWarning:

Mean of empty slice

C:\Users\Roni\AppData\Local\Temp\ipykernel_15164\3004972900.py:13: RuntimeWarning:

Mean of empty slice

C:\Users\Roni\AppData\Local\Temp\ipykernel_15164\3004972900.py:13: RuntimeWarning:

Mean of empty slice

c:\Users\Roni\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.



[1 2 2 2 1 2 0 2 1 1 1 1 2 2 2 2 2 2 1 0 2 1 0 2 2 2 2 0 2 1 2 1 2 1 0 2 1
 2 2 2 2 1 1 2 2 2 1 1 2 1 2 2 2 2 2 2 1 1 2 0 2 2 2 2 2 1 2 2 2]


array([[ 8.26905281e-06, -1.33426915e-06, -1.00000000e+00],
       [-9.99983446e-01,  5.75392155e-03, -8.27659320e-06]])

In [110]:

fig = go.Figure()

for lbl in np.unique(kmeans.labels_):
    exp_names = np.unique(feat.iloc[kmeans.labels_==lbl]['exp_name'])
    feat_lbl = feat.iloc[kmeans.labels_==lbl]
    for idx,exp_name in enumerate(exp_names): 
        mov_exp_lbl = list(feat_lbl.loc[feat_lbl['exp_name'] == exp_name]['mov'])

        prop = 'pitch_y_frame_dot'
        prop = 'CM_dot_dot_projected'
        # prop = 'pitch_body'

        color_map = colormap.datad["tab10"]['listed']

        experiments[exp_name].plot_prop_movies(prop,'body',fig,mov = mov_exp_lbl,case = 'plot_exp',prop_x = 'time',color = color_map[lbl],legend = f'{exp_name}_cluster{lbl}')
fig.show()
# feat.iloc[kmeans.labels_==lbl]['exp_name'] == exp_names[0]


In [62]:
feat_lbl[feat_lbl['exp_name'] == exp_name]

,index,pitch_body,yaw_body,roll_body,pitch_body_dot,yaw_body_dot,roll_body_dot,pitch_body_dot_dot,yaw_body_dot_dot,roll_body_dot_dot,...,CM_dot_projected,CM_dot_dot_projected,p,q,r,p_dot,q_dot,r_dot,exp_name,mov


In [353]:
from scipy import stats


exp = experiments[list(experiments.keys())[0]]
prop = 'pitch_y_frame'
prop = 'q'
prop = 'CM_dot_dot_projected'
props = ['pitch_y_frame_dot_dot','pitch_y_frame_dot']

window = (73*3)//2
fig,ax = plt.subplots(2,1,sharex=True)
max_prop = {}
for idx,prop in enumerate(props):
    max_prop[prop] = [] 
    for exp in experiments.values():
        mov_to_keep = []
        acc_mean,time_inter = exp.mean_time_series_prop(prop,'body')
        time_to_check = (time_inter > 0 ) & (time_inter <100 )
        time_for_max = time_inter[time_to_check]
        max_prop[prop].append(time_for_max[np.nanargmax(acc_mean[time_to_check])])
        ax[idx].plot(time_inter,acc_mean)

g:\Documents\experiment_analysis\data_analysis\Experiment.py:101: RuntimeWarning:

Mean of empty slice



In [355]:
max_prop['pitch_y_frame_dot_dot']
prop = 'pitch_y_frame_dot_dot'

acc_mean,time_inter = exp.mean_time_series_prop(prop,'body')
idx0 = max_prop['pitch_y_frame_dot_dot'][0]
idx1 = max_prop['pitch_y_frame_dot'][0]



prop = 'pitch_y_frame_dot_dot'
exp = experiments[list(experiments.keys())[0]]
import matplotlib.cm as colormap

color_map = colormap.datad["tab10"]['listed']
fig = go.Figure()
# exp.plot_prop_movies('pitch_y_frame_dot_dot','body',0,fig,mov = False,case = 'plot_mov',prop_x = 'time')
time_max = []
for idx,mov_name in enumerate(exp.mov_names):
    mov = exp.get_mov(mov_name)

    time = mov.get_prop('time','body')[:,0]

    idx_time = (time >idx0 -10 ) & (time < idx1+10) if (idx0 != False) & (idx1 != False) else (time >-np.inf ) & (time < np.inf)

    acc = mov.get_prop(prop,'body')[idx_time,0]
    time = mov.get_prop('time','body')[idx_time,0]


    idx_max = np.argmax(acc)

    # idx2 = np.where(time == min_max_pitch[1][idx])[0][0]
    color = color_map[idx%len(color_map)]
    fig.add_trace(go.Scattergl(x=[time[idx_max]],
                    y=[acc[idx_max]],
                    marker=dict(
                        color=f'rgba{str(tuple(np.append(color,1)))}',
                        size=10,
                    ),
                legendgroup = mov_name,
                showlegend = False))
    time_max.append(time[idx_max])
fig.show()


# plt.figure()
# min_max_pitch =[exp.min_max_point_movies('pitch_y_frame_dot_dot') for exp in experiments]
plt.figure()
plt.hist(time_max ,alpha = 0.5,bins = 40,label =exp.experiment_name, range = (0,100) ) 





(array([0., 0., 0., 0., 0., 0., 0., 6., 5., 2., 3., 5., 4., 5., 2., 0., 0.,
        1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.]),
 array([  0. ,   2.5,   5. ,   7.5,  10. ,  12.5,  15. ,  17.5,  20. ,
         22.5,  25. ,  27.5,  30. ,  32.5,  35. ,  37.5,  40. ,  42.5,
         45. ,  47.5,  50. ,  52.5,  55. ,  57.5,  60. ,  62.5,  65. ,
         67.5,  70. ,  72.5,  75. ,  77.5,  80. ,  82.5,  85. ,  87.5,
         90. ,  92.5,  95. ,  97.5, 100. ]),
 <BarContainer object of 40 artists>)

In [17]:
from sklearn.decomposition import PCA
import pandas as pd

from sklearn.cluster import KMeans

mov_feat = []
exp = experiments[4]

exp_pd = []
for exp in experiments[0:1]:
    mov_feat = []
    for idx,mov_name in enumerate(exp.mov_names):
        mov = exp.get_mov(mov_name)
        mov_feat.append(np.nanmean(mov.data['body'][mov.ref_frame:mov.ref_frame +200*16  ,:],axis = 0))

    featpd_exp = pd.DataFrame(data = mov_feat,columns = mov.header['body'])
    featpd_exp['exp_name'] = exp.experiment_name
    featpd_exp['mov'] = exp.mov_names
    exp_pd.append(featpd_exp)

feat = pd.concat(exp_pd,axis = 0).dropna().reset_index()


props = ['CM_dot_projected','pitch_body','pitch_body_dot','pitch_y_frame']
dataforpca = feat[props]
n_comp = 4
pca = PCA(n_components=n_comp)
principalComponents = pca.fit_transform(dataforpca)
principalDf = pd.DataFrame(data = principalComponents
             , columns = [f'pca_{idx}' for idx in range(n_comp)])



featpd = pd.concat([feat, principalDf], axis = 1)
kmeans = KMeans(n_clusters=2, random_state=0, n_init="auto").fit(featpd[pca_name])
print(kmeans.labels_)


[0 1 0 0 0 0 0 1 0 0 1 0 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 0 1 0 1 1 1]


c:\Users\Roni\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.



In [14]:
exp = experiments[0]
mov_name1 = [exp.mov_names[idx] for idx in np.where(kmeans.labels_==0)[0]]
mov_name2 = [exp.mov_names[idx] for idx in np.where(kmeans.labels_==1)[0]]
# mov_name3 = [exp.mov_names[idx] for idx in np.where(labels==2)[0]]

prop = 'pitch_body'
prop = 'CM_dot_projected'
# prop = 'pitch_body'

color_map = colormap.datad["tab10"]['listed']

fig = go.Figure()
exp.plot_prop_movies(prop,'body',fig,mov = mov_name1,case = 'plot_exp',prop_x = 'time',color = color_map[0],legend = 'custer1')
exp.plot_prop_movies(prop,'body',fig,mov = mov_name2,case = 'plot_exp',prop_x = 'time',color = color_map[1],legend='cluster2')
# exp.plot_prop_movies(prop,'body',fig,mov = mov_name3,case = 'plot_exp',prop_x = 'time',color = color_map[2],legend='cluster3')


fig.show()


In [4]:

pca_name = [f'pca_{idx}' for idx in range(pca.n_components_)]
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0, n_init="auto").fit(featpd[pca_name])
print(kmeans.labels_)
plt.plot(featpd[pca_name[0]][kmeans.labels_==0],featpd[pca_name[1]][kmeans.labels_==0],'*')
plt.plot(featpd[pca_name[0]][kmeans.labels_==3],featpd[pca_name[1]][kmeans.labels_==3],'*')


pca_color = ['red','blue','green','pink','yellow','gray']
prop = 'pitch_y_frame'

for lbl in  np.unique(kmeans.labels_):
    mov_name = [exp.mov_names[idx] for idx in np.where(kmeans.labels_==lbl)[0]]
    for mov in  mov_name:
        mov1 = exp.get_mov(mov)
        plt.plot(mov1.get_prop(prop,'body'),pca_color[lbl])



[0 0 1 1 1 1 1 1 0 1 1 1 0 1 0 1 0 1 1 0 1 1 1 0 1 0 1 0 0 0 1 1 0 1]


c:\Users\Roni\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.



In [253]:
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
props = ['CM_dot_projected','pitch_body','pitch_body_dot','pitch_y_frame','pitch_y_frame_dot','pitch_y_frame_dot_dot','CM_real_x_body_dot_smth','CM_real_z_body_dot_smth']

linkage_data = linkage(featpd[props], method='ward', metric='euclidean')
dendrogram(linkage_data)

hierarchical_cluster = AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='ward')

labels = hierarchical_cluster.fit_predict(featpd[props])

labels

c:\Users\Roni\anaconda3\lib\site-packages\sklearn\cluster\_agglomerative.py:983: FutureWarning:

Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead



array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int64)

In [235]:

fig = go.Figure()
prop = props[0]
mov_name1 = [exp.mov_names[idx] for idx in np.where(labels==0)[0]]

exp.plot_prop_movies(prop,'body',fig,case = 'plot_mov',prop_x = 'time',mov = mov_name1)

fig.show()


In [5]:
exp = experiments[0]
mov_name1 = [exp.mov_names[idx] for idx in np.where(kmeans.labels_==0)[0]]
mov_name2 = [exp.mov_names[idx] for idx in np.where(kmeans.labels_==1)[0]]
# mov_name3 = [exp.mov_names[idx] for idx in np.where(labels==2)[0]]

prop = 'pitch_y_frame_dot'
prop = 'CM_dot_dot_projected'
# prop = 'pitch_body'

color_map = colormap.datad["tab10"]['listed']

fig = go.Figure()
exp.plot_prop_movies(prop,'body',fig,mov = mov_name1,case = 'plot_exp',prop_x = 'time',color = color_map[0],legend = 'custer1')
exp.plot_prop_movies(prop,'body',fig,mov = mov_name2,case = 'plot_exp',prop_x = 'time',color = color_map[1],legend='cluster2')
# exp.plot_prop_movies(prop,'body',fig,mov = mov_name3,case = 'plot_exp',prop_x = 'time',color = color_map[2],legend='cluster3')


fig.show()


In [34]:
kmeans.labels_

array([0, 0, 0, 0, 5, 0, 0, 5, 0, 5, 1, 5, 0, 0, 0, 5, 5, 5, 5, 5, 0, 6,
       5, 4, 0, 0, 0, 6, 0, 0, 0, 0, 5, 5, 5, 0, 4, 4, 2, 5, 5, 5, 3, 5])

In [375]:
from scipy import stats


exp = experiments[0]
prop = 'pitch_y_frame'
prop = 'q'
prop = 'CM_dot_dot_projected'

window = (73*3)//2
fig,ax = plt.subplots(2,1,sharex=True,sharey=True)
for exp in experiments:
    mov_to_keep = []
    acc_mean,time_inter = exp.mean_time_series_prop(prop,'body')

    for mov_name in exp.mov_names[:]:
        mov = exp.get_mov(mov_name)
        acc = mov.get_prop(prop,'body')
        time = mov.get_prop('time','body')
        time_to_sub,time_inter_idx,time_idx = np.intersect1d(time_inter,time,return_indices = True)

        dist = acc_mean[time_inter_idx] - acc[time_idx,0]
        # if (np.abs(np.nansum(acc_mean[time_inter_idx] - acc[time_idx,0])/np.nansum(acc[time_idx,0])) < 2) & (np.nanmax(dist) < 8):
        if (np.nanmax(dist) < 10) & (np.abs(np.nanmin(dist)) < 10):
            mov_to_keep.append(mov_name)
            # ax[0].plot(time[time_idx],dist.T,'gray')
            # ax[1].plot(time,acc,'gray')
    # exp.mov_names = mov_to_keep
            

    idx = np.where(np.isfinite(time_inter) & np.isfinite(acc_mean))[0]
    idx0 = 200
    idx1 = 1000
    p = np.poly1d(np.polyfit(time_inter[idx[idx0:idx1]], acc_mean[idx[idx0:idx1]], 4, rcond=None, full=False, w=None, cov=False)   )     
    xp = time[idx0:idx1]
    plt.plot(time_inter[idx],acc_mean[idx],linewidth = 6)
    # plt.plot(time_inter[idx[idx0:idx1]], acc_mean[idx[idx0:idx1]], '.', xp, p(xp), '-')
    # p = np.poly1d(np.polyfit(time[idx0:idx1,0].T, acc[idx0:idx1,0].T, 4, rcond=None, full=False, w=None, cov=False)   )     

    # plt.plot(time[window:-1,0],acc[window:-1,0], '.', xp, p(xp), '-',linewidth = 3)


In [313]:
x = time[idx0:idx1]
p = np.polyfit(time[idx0:idx1,0].T, acc[idx0:idx1,0].T, 4, rcond=None, full=False, w=None, cov=False)
plt.plot(x,p[0]*x**4+p[1]*x**3+p[2]*x**2+p[3]*x+p[4])

plt.plot(4*p[0]*x**3+3*p[1]*x**2+2*p[2]*x+p[3])

In [367]:
mov_name = 'mov111'
mov = exp.get_mov(mov_name)
acc = mov.get_prop('pitch_y_frame_dot_dot','body')
cm_dot = mov.get_prop('CM_dot_projected','body')
time = mov.get_prop('time','body')

fig,ax = plt.subplots(2,1)
ax[0].plot(time,cm_dot)
ax[1].plot(time,acc)
plt.figure()
min_max_pitch =[exp.min_max_point_movies('q') for exp in experiments]
[plt.hist(min_max[1],alpha = 0.5,bins = 20,label =exp.experiment_name, range = (0,100) ) for min_max,exp in zip(min_max_pitch,experiments)]


KeyError: 'mov111'

In [428]:
min_max_v = [exp.min_max_point_movies('pitch_y_frame_dot_dot') for exp in experiments[0:4]]
plt.figure()
[plt.hist(min_max[1],alpha = 0.5,bins = 40,label =exp.experiment_name ) for min_max,exp in zip(min_max_v,experiments[0:4])]


[(array([1., 0., 0., 0., 0., 1., 3., 7., 8., 5., 4., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 1., 1., 2., 0., 0., 0., 0., 1., 0., 1., 1., 2.,
         1., 1., 1., 0., 0., 2.]),
  array([-10.5625,  -5.7875,  -1.0125,   3.7625,   8.5375,  13.3125,
          18.0875,  22.8625,  27.6375,  32.4125,  37.1875,  41.9625,
          46.7375,  51.5125,  56.2875,  61.0625,  65.8375,  70.6125,
          75.3875,  80.1625,  84.9375,  89.7125,  94.4875,  99.2625,
         104.0375, 108.8125, 113.5875, 118.3625, 123.1375, 127.9125,
         132.6875, 137.4625, 142.2375, 147.0125, 151.7875, 156.5625,
         161.3375, 166.1125, 170.8875, 175.6625, 180.4375]),
  <BarContainer object of 40 artists>),
 (array([ 3.,  1.,  0.,  0.,  7., 10., 11., 10.,  3.,  2.,  1.,  0.,  2.,
          0.,  0.,  1.,  3.,  3.,  5.,  2.,  6.,  2.,  5.,  2.,  1.,  0.,
          1.,  7.,  1.,  2.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,
          1.]),
  array([-17.875    , -10.0421875,  -2.209375 ,   5.623437

In [99]:
exp = experiments[3]
mov = exp.get_mov('mov101')


for mov_name in exp.mov_names[0:20]:
    mov = exp.get_mov(mov_name)

    v = mov.get_prop('CM_real_x_body_smth','body',three_col=3)
    cm = mov.get_prop('CM_real_x_body_smth','body',three_col=3)
    cm[:,2] = 0
    v[:,2] = 0

    cm = cm - cm[mov.ref_frame]

    vax = v[mov.ref_frame,:]/np.linalg.norm(v[mov.ref_frame,:])
    angle_to_rotate = np.arctan2(vax[1],vax[0])
    x_rotated = cm[:,0]*np.cos(angle_to_rotate) + cm[:,1]*np.sin(angle_to_rotate)
    y_rotated = cm[:,0]*np.sin(angle_to_rotate) - cm[:,1]*np.cos(angle_to_rotate)


    plt.plot(x_rotated,y_rotated)



In [19]:
exp = experiments[3]

time_for_mean = 30
num_std = 4
winsow_size = (73*7)//2
plot_all = True
prop = 'CM_dot_dot_projected'
add_std = False
sub_std = True
# fig,ax = plt.subplots(len(experiments),1)
plt.figure()
for idx,exp in enumerate(experiments[4:5]):
    time_onset = []

    for mov_name in exp.mov_names:
        mov = exp.get_mov(mov_name) 
        acc = mov.get_prop(prop,'body')[:,0]


        time = mov.get_prop('time','body')[:,0]
        try:
            end_time = np.where(time == time_for_mean)[0][0]
        except:
            continue
        # acc = acc - np.mean(acc[winsow_size:end_time])

        mean_acc = np.mean(acc[winsow_size:end_time])
        std_acc = np.std(acc[winsow_size:end_time])
        if len(np.where(acc[end_time:-1] < (mean_acc - np.abs(num_std*std_acc)))[0]) > 0:
            if add_std == True: 
                idx_onset = np.where(acc[end_time:-1] > (mean_acc + np.abs(num_std*std_acc)))[0][0]
            if sub_std == True:
                idx_onset = np.where(acc[end_time:-1] < (mean_acc - np.abs(num_std*std_acc)))[0][0]

            time_onset.append(time[end_time:][idx_onset])
            
            if plot_all == True:
                # plt.figure()
                plt.plot(time[winsow_size:],acc[winsow_size:])
                plt.scatter(time[end_time:][idx_onset],acc[end_time:][idx_onset])



                plt.vlines(time[end_time],-0.2,0.5,'red')
                # plt.vlines(time[winsow_size],-3,5,'red')

                # plt.vlines(time[end_time:][idx_onset],-3,5,'green')
                # plt.hlines(mean_acc,-10,200,'gray')
                # plt.hlines(mean_acc + std_acc,-10,200,'black')
                # plt.hlines(mean_acc - std_acc,-10,200,'black')

    
#     ax[idx].hist(time_onset,bins = 30,range = (time_for_mean,80))
#     ax[idx].set_title(exp.pertubation_name)
# plt.title(exp.pertubation_name)


In [10]:
end_time - winsow_size

234

In [187]:


plt.figure()
plt.plot(time,acc)
plt.vlines(time[end_time],-3,2,'red')
plt.vlines(time[winsow_size],-3,2,'red')

plt.vlines(time[end_time:][idx_onset],-3,2,'green')
plt.hlines(mean_acc,-10,200,'gray')
plt.hlines(mean_acc + std_acc,-10,200,'black')
plt.hlines(mean_acc - std_acc,-10,200,'black')

In [ ]:
zero_acc = [exp.zero_velocity_movies('CM_dot_dot_projected') for exp in experiments]
new_zer = []
for zero_acc_exp in zero_acc:
    zero = []
    for zer in zero_acc_exp:
       
        if len(zer) > 1:
            zero.append(zer[1])
        else: 
            zero.append(zer[0])
    new_zer.append(zero)


In [ ]:

plt.figure()
[plt.hist(np.array(new_zer[idx]),alpha=0.5,bins = 20,label = exp.experiment_name) for idx,exp in enumerate(experiments[0:5]) if exp.pertubation != False]
plt.legend()
plt.title('zero acc')



Text(0.5, 1.0, 'diff max pitch min v')

In [ ]:
from scipy.signal import argrelextrema, savgol_filter,find_peaks

x = np.array(range(80))
data = (x - 50)**3  + (np.random.rand(1,len(x)) - 0.5)[0]*10000


smoothed = savgol_filter(data,31,2,deriv = 1)
smoothed2 = savgol_filter(data,31,2,deriv = 2) 
fig,ax = plt.subplots(3,1)
ax[0].plot(data)
ax[1].plot(smoothed)
ax[2].plot(smoothed2)



In [3]:
import matplotlib.cm as colormap
color_map = colormap.datad["tab10"]['listed']

exp = experiments[3]

fig = go.Figure()
[exp.plot_prop_movies('pitch_y_frame_dot','body',fig,mov = False,legend = exp.pertubation_name,case = 'plot_exp',mode = 'lines',color = color_map[idx + 2]) for idx,exp in enumerate(experiments)]
fig.show()

# fig = go.Figure()
# exp.plot_prop_movies('pitch_y_frame_dot','body',color_map[0],fig,mov = False,case = 'plot_mov',prop_x = 'time')
# fig.show()

# fig = go.Figure()
# exp.plot_prop_movies('roll_body','body',color_map[0],fig,mov = False,case = 'plot_mov')
# fig.show()




In [ ]:
import matplotlib.cm as colormap
exp = experiments[3]

# exp.plot_movie_prop('time','v_xy', case = 'plot_exp',prop = 'body', 
#                      fig = fig, mov = False,mode = 'markers') 
# fig.show()
color_map = colormap.datad["tab10"]['listed']
fig = go.Figure()
exp.plot_prop_movies('CM_dot_dot_projected','body',color_map[0],fig,mov = False,case = 'plot_mov')
fig.show()




fig = go.Figure()
[exp.plot_prop_movies('CM_dot_projected','body',color_map[idx + 2],fig,mov = False,case = 'plot_exp',mode = 'lines') for idx,exp in enumerate(experiments)]
fig.show()


fig = go.Figure()
[exp.plot_prop_movies('CM_dot_dot_projected','body',color_map[idx + 2],fig,mov = False,case = 'plot_exp',mode = 'lines') for idx,exp in enumerate(experiments)]
fig.show()

fig = go.Figure()
[exp.plot_prop_movies('pitch_y_frame','body',color_map[idx + 2],fig,mov = False,case = 'plot_exp',mode = 'lines') for idx,exp in enumerate(experiments)]
fig.show()


fig = go.Figure()
exp.plot_prop_movies('q','body',color_map[0],fig,mov = False,case = 'plot_mov',mode = 'lines') 
fig.show()


fig = go.Figure()
exp.plot_prop_movies('pitch_y_frame_dot','body',color_map[0],fig,mov = False,case = 'plot_mov',mode = 'lines') 
fig.show()

fig = go.Figure()
exp.plot_prop_movies('pitch_y_frame_dot_dot','body',color_map[0],fig,mov = False,case = 'plot_mov',mode = 'lines') 
fig.show()


fig = go.Figure()
[exp.plot_prop_movies('CM_dot_dot_projected','body',color_map[idx + 2],fig,mov = False,case = 'plot_exp',mode = 'lines') for idx,exp in enumerate(experiments)]
fig.show()

In [ ]:
import numpy as np



plt.figure()
delta_zero = [plt.hist(np.array(zero_velocity[idx]) ,alpha=0.5,bins = 20,label = exp.experiment_name) for idx,exp in enumerate(experiments) if exp.pertubation != False]
plt.title('time of zero velocity')
plt.legend()

plt.figure()
[plt.hist(np.array(min_max_v[idx][0]) ,alpha=0.5,bins = 20,label = exp.experiment_name) for idx,exp in enumerate(experiments) if exp.pertubation != False]
plt.legend()
plt.title('time of minimal velocity')


plt.figure()
[plt.hist(np.array(min_max_pitch[idx][1]),alpha=0.5,bins = 15,label = exp.experiment_name,range=(50, 120)) for idx,exp in enumerate(experiments) if exp.pertubation != False]
plt.legend()
plt.title('time of maximal pitch')


plt.figure()
[plt.hist(np.array(min_max_v[idx][0] - min_max_pitch[idx][1] ),alpha=0.5,bins = 20,label = exp.experiment_name) for idx,exp in enumerate(experiments[0:2]) if exp.pertubation != False]
plt.legend()
plt.title('diff max pitch min v')




Text(0.5, 1.0, 'diff max pitch min v')

mean of all lines. normalized: data - mean(data(0:t == 0))
velocity
pitch
mean mean phi

- difference in peek between pulses - velocity and pitch
- pitch - long time reaction - above 100 ms
- (mean(phi)L + mean(phi)R)/2 - visible peek at ~55
- peek around 45 pitch_dot
- peek around 60 - p 

maybe a better way to do it? 


In [ ]:
exp = experiments[-1]
exp.mov_names
mov = exp.get_mov('mov10')
mov.header['body'].keys()

dict_keys(['pitch_body', 'yaw_body', 'roll_body', 'pitch_body_dot', 'yaw_body_dot', 'roll_body_dot', 'pitch_body_dot_dot', 'yaw_body_dot_dot', 'roll_body_dot_dot', 'frames', 'time', 'CM_real_x_body', 'CM_real_y_body', 'CM_real_z_body', 'phi_rw_max_idx', 'phi_rw_max_val', 'phi_lw_max_idx', 'phi_lw_max_val', 'phi_rw_min_idx', 'phi_rw_min_val', 'phi_lw_min_idx', 'phi_lw_min_val', 'yaw_z_frame', 'pitch_y_frame', 'roll_x_frame', 'yaw_z_frame_dot', 'pitch_y_frame_dot', 'roll_x_frame_dot', 'yaw_z_frame_dot_dot', 'pitch_y_frame_dot_dot', 'roll_x_frame_dot_dot', 'CM_real_x_body_smth', 'CM_real_y_body_smth', 'CM_real_z_body_smth', 'CM_real_x_body_dot_smth', 'CM_real_y_body_dot_smth', 'CM_real_z_body_dot_smth', 'CM_real_x_body_dot_dot_smth', 'CM_real_y_body_dot_dot_smth', 'CM_real_z_body_dot_dot_smth', 'CM_dot_projected', 'p', 'q', 'r', 'p_dot', 'q_dot', 'r_dot', 'p_dot_dot', 'q_dot_dot', 'r_dot_dot'])

In [366]:
N = 15
dt = 1/16000
window = (73*7)//2
# prop = 'pitch_y_frame'
prop = 'CM_xy_dot_dot'
# prop = 'mean_mean_phi'
wing_body = 'mean_wing'
wing_body = 'body'
# prop = 'pitch_y_frame_dot'
prop = 'CM_dot_projected'
prop = 'q'

# prop = 'q'
# prop = 'pitch_y_frame_dot_dot'

conv = False

plt.figure()



for exp,color in zip(experiments[:],['blue','red','green','pink','orange','yellow','black']):
    t_list = np.arange(-40,400,dt*1000)
    cmlist = np.full((len(exp.mov_names),len(t_list)),np.nan)
    for idx_mov,mov_name in enumerate(exp.mov_names):
        mov = exp.get_mov(mov_name)
        time = mov.get_prop('time',wing_body)[window:,0]
        start_end_time = np.where(np.isin(time , t_list))[0]

        cm_dot = mov.get_prop(prop,wing_body)[window:,0]
        cm_dot = cm_dot - np.mean(cm_dot[window:mov.ref_frame + 1])
        time_inter,tlist_idx,time_idx = np.intersect1d(t_list,time,return_indices = True )
        cmlist[idx_mov,tlist_idx] = cm_dot[time_idx]


    
        
    plt.plot(t_list,np.nanmean(cmlist,axis = 0),color = color,linestyle='None',
             label = exp.experiment_name,linewidth = 2,marker = 'o',markersize = 3)
#     plt.hlines(0,t_list[0],t_list[-1],color = 'black')
# plt.legend()
from tslearn.clustering import TimeSeriesKMeans

model = TimeSeriesKMeans(n_clusters=3, metric="dtw",
                         max_iter=10)
model.fit(cmlist)


c:\Users\Roni\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning:

Mean of empty slice.

c:\Users\Roni\anaconda3\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning:

invalid value encountered in double_scalars

C:\Users\Roni\AppData\Local\Temp\ipykernel_14100\406941317.py:38: RuntimeWarning:

Mean of empty slice



ValueError: One of the input time series contains only nans or has zero length.

plot velocity, pitch and p per movie

In [230]:
from tslearn.clustering import TimeSeriesKMeans

prop_exp = []
for exp in experiments[0:3]:
    for mov_name in exp.mov_names[0:10]:
        mov = exp.get_mov(mov_name)
        prop.append(mov.get_prop('CM_dot_dot_projected','body')[window:2000,:])

        wak = np.hstack(prop)

    wakka = np.vstack(wak)

model = TimeSeriesKMeans(n_clusters=len(experiments[0:3]), metric="dtw",
                         max_iter=10)
y = model.fit(wakka)


KeyboardInterrupt: 

In [224]:

prop_exp = []
for exp in experiments[:]:
    prop = []
    for mov_name in exp.mov_names:
        mov = exp.get_mov(mov_name)
        prop.append(mov.get_prop('CM_dot_dot_projected','body')[window:2000,:])

        wak = np.hstack(prop)

    wakka = np.vstack(wak)

In [225]:
wakka

array([[ 1.0985002 , -0.89911951,  0.02226098, ...,  0.04350475,
         0.84437364,  0.12369192],
       [ 1.09910101, -0.900818  ,  0.02050402, ...,  0.0433696 ,
         0.85254303,  0.12260795],
       [ 1.10099964, -0.90065647,  0.01956409, ...,  0.04388182,
         0.86069173,  0.12296666],
       ...,
       [-0.67824592, -2.05947014,  0.888622  , ..., -0.43594607,
        -0.31557689,  0.46349992],
       [-0.67639865, -2.06800012,  0.88757633, ..., -0.4377551 ,
        -0.31451646,  0.46189768],
       [-0.67455139, -2.07653009,  0.88575382, ..., -0.43952561,
        -0.31464514,  0.46200124]])

In [ ]:
plt.figure()
exp_name =['manipulated_2023_08_10_100ms',
           'manipulated_2023_08_09_60ms',
           'manipulated_2022_02_03_dark',
           'manipulated_2023_08_06_40ms',
           'manipulated_2023_08_09_80ms',
           'manipulated_2023_08_07_10ms',
           'manipulated_2023_08_07_5ms',
            # 'manipulated_2022_01_31_darkdown',
] 
for exp in  experiments:
    for mov,color in zip(exp.mov_names[:],['red']*len(exp.mov_names)):#,'blue','green','black']):
        mov = exp.get_mov(mov)
        cm_xy_dot_dot = mov.get_prop('CM_dot_projected','body')
        # cm_xy_dot = mov.get_prop('CM_dot_projected','body')
        time = mov.get_prop('time','body')

        # pitch = mov.get_prop('pitch_y_frame','body')/100
        p = mov.get_prop('pitch_body','body')

        # plt.plot(time,cm_xy_dot_dot,color, linestyle = 'dashed')
        # plt.plot(time,cm_xy_dot,color, linestyle = '--')

        # plt.plot(time,pitch,color)
        # plt.plot(time,p,color,linestyle=':')
        plt.plot(cm_xy_dot_dot,p,color,linestyle=':')

plt.xlabel('cm_dot_dot')
plt.ylabel('pitch angle')

# plt.hlines(45,-15,10,color = 'black')



Text(0, 0.5, 'pitch angle')

In [ ]:
exp = experiments[0]

fig = go.Figure()
exp.plot_movie_prop('time','roll_body', case = 'plot_mov',prop = 'body', fig = fig, mov = False,add_horizontal_line = 0)
fig.show()

In [ ]:

exp = experiments[3]
mov = exp.get_mov('mov104')
mov.plot_3d_traj_movie('CM_dot_dot_projected')

In [ ]:

exp = experiments[1]
mov = exp.get_mov('mov12')
mov.plot_3d_traj_movie('CM_dot_dot_projected')

In [ ]:

exp = experiments[0]
mov = exp.get_mov('mov130')
mov.plot_3d_traj_movie('CM_dot_dot_projected')

In [ ]:
exp = experiments[0]
mov = exp.get_mov('mov3')
mov.header['body']

{'pitch_body': 0,
 'yaw_body': 1,
 'roll_body': 2,
 'pitch_body_dot': 3,
 'yaw_body_dot': 4,
 'roll_body_dot': 5,
 'pitch_body_dot_dot': 6,
 'yaw_body_dot_dot': 7,
 'roll_body_dot_dot': 8,
 'frames': 9,
 'time': 10,
 'CM_real_x_body': 11,
 'CM_real_y_body': 12,
 'CM_real_z_body': 13,
 'phi_rw_max_idx': 14,
 'phi_rw_max_val': 15,
 'phi_lw_max_idx': 16,
 'phi_lw_max_val': 17,
 'phi_rw_min_idx': 18,
 'phi_rw_min_val': 19,
 'phi_lw_min_idx': 20,
 'phi_lw_min_val': 21,
 'CM_xy': 22,
 'CM_xy_dot': 23,
 'CM_xy_dot_dot': 24,
 'yaw_z_frame': 25,
 'pitch_y_frame': 26,
 'roll_x_frame': 27,
 'yaw_z_frame_dot': 28,
 'pitch_y_frame_dot': 29,
 'roll_x_frame_dot': 30,
 'yaw_z_frame_dot_dot': 31,
 'pitch_y_frame_dot_dot': 32,
 'roll_x_frame_dot_dot': 33,
 'mean_idx': 34,
 'p': 35,
 'q': 36,
 'r': 37,
 'p_dot': 38,
 'q_dot': 39,
 'r_dot': 40,
 'p_dot_dot': 41,
 'q_dot_dot': 42,
 'r_dot_dot': 43}

In [ ]:

fig = go.Figure()
exp.plot_movie_prop('time','p', case = 'plot_mov',prop = 'body', fig = fig, mov = False,add_horizontal_line = 0)
fig.show()


In [ ]:
import plotly.graph_objects as go
import plotly

exp = experiments[0]
exp.pqr_movies() 
exp.add_mean_prop_movies('q','body','mean_wing') 
exp.mean_mean_props_movies('phi_rw_min_val','phi_lw_min_val','mean_wing','mean_front_phi')
exp.mean_mean_props_movies('phi_rw','phi_lw','mean_wing','mean_mean_phi')

fig = go.Figure()
exp.plot_movie_prop('time','q', case = 'plot_mean',prop = 'mean_wing', fig = fig, mov = False,add_horizontal_line = 0)
fig.show()

fig = go.Figure()

exp.plot_movie_prop('time','p', case = 'plot_mov',prop = 'body', fig = fig, mov = False,add_horizontal_line = 0)
fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()    
[fig.add_trace(go.Histogram(x=np.hstack(min), name=exp.experiment_name, nbinsx=30)) for min,exp in zip(zero_velocity,experiments)]
fig.update_layout(title = 'time of minimum velocity')
fig.update_traces(opacity = 0.9)
fig.show()


Save plots to folders

In [ ]:
import plotly.graph_objects as go
import plotly
import matplotlib.cm as colormap
color_map = colormap.datad["tab10"]['listed']



def plot_and_save_movie_prop(exp,yprop,wing_body,add_horizontal_line = False):
    
    fig = go.Figure()

    exp.plot_prop_movies(yprop,'body',color_map[0],fig,mov = False,case = 'plot_mov')
    fig.show()
    plotly.offline.plot(fig, filename=f'{exp.figures_path}/{yprop}_plot.html',auto_open=False)




for exp in experiments:
    try:
        exp.plot_3d_traj_movies('CM_dot_dot_projected',save_plot = True, mov = False)

        plot_and_save_movie_prop(exp,yprop = 'CM_dot_dot_projected',wing_body = 'body',add_horizontal_line = 0)
        plot_and_save_movie_prop(exp,yprop = 'pitch_y_frame',wing_body = 'body',add_horizontal_line = 0)
        plot_and_save_movie_prop(exp,yprop = 'pitch_y_frame_dot',wing_body = 'body',add_horizontal_line = 0)
        plot_and_save_movie_prop(exp,yprop = 'CM_dot_projected',wing_body = 'body',add_horizontal_line = 0)
    except:
        continue
    # exp.mean_mean_props_movies('phi_rw_min_val','phi_lw_min_val','mean_wing','mean_front_phi')
    # exp.mean_mean_props_movies('phi_rw','phi_lw','mean_wing','mean_mean_phi')

    # plot_and_save_movie_prop(exp,yprop = 'mean_front_phi',wing_body = 'mean_wing')
    # plot_and_save_movie_prop(exp,yprop = 'mean_mean_phi',wing_body = 'mean_wing')

